In [1]:
# Welcome, To ~ 
#   The Hitman's car_______"(0TYT=0> ______________version-10
#___________________________________________________________

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install ultralytics
!pip install deepface
!pip install cmapy
!pip install timm

In [10]:
# ~~ Setting Everything Up /...
#________________________________________________________________________________________________
#source_vid_path = "/content/drive/MyDrive/Final-Task/Data/Obama-db/obamawalk.mp4"
#source_vid_path = "data/workshop-vid.mp4"
#source_vid_path = "data/obamawalk.mp4"
source_vid_path = 0
#________________________________________________________________________________________________
#vid_save_path = "/content/drive/MyDrive/Final-Task/Data/Obama-db/obama-output.mp4"
vid_save_path = "outputs/my_output.mp4"
#________________________________________________________________________________________________
#target_face_path = "/content/drive/MyDrive/Final-Task/Data/Obama-db/obama2.jpg"
#target_face_path = "obama2.jpg"
target_face_path = "me.jpg"
#________________________________________________________________________________________________
#Sensitivity to recognized person's movements
#sensitivity = 0.001    
sensitivity = 0.0001
#________________________________________________________________________________________________

In [2]:
import os
import time
import cv2 as cv
import numpy as np
import torch
from ultralytics import YOLO
from deepface import DeepFace
from deepface.commons import functions
from deepface.detectors import FaceDetector
from matplotlib import pyplot as plt
import cmapy
#from facenet_pytorch import InceptionResnetV1, MTCNN

In [3]:
# ~~ The deepface package ~~

#Face detection ~
backends = [
  'opencv', 
  'ssd', 
  'dlib', 
  'mtcnn', 
  'retinaface', 
  'mediapipe'
]

#Face recognition ~
models = [
  "VGG-Face", 
  "Facenet", 
  "Facenet512", 
  "OpenFace", 
  "DeepFace", 
  "DeepID", 
  "ArcFace", 
  "Dlib", 
  "SFace",
]

In [4]:
# ~~ The Models used in this project ~~

#Objct detection - to detect people---------------------------------------------#
#seg_model = YOLO('yolov8n-seg.pt')
yolo_model = YOLO('yolov8n.pt')

#Face detection ~ --------------------------------------------------------------#
face_detector = FaceDetector.build_model('ssd')
'''face_detector = MTCNN(
    image_size=299, 
    margin=40, 
    min_face_size=20,
    keep_all=False,
    )'''

#Face recognition ~ -----------------------------------------------------------#
DeepFace.build_model('SFace')
DeepFace.build_model('Facenet512')
#face_recognizer = InceptionResnetV1(pretrained='vggface2').eval()



100%|██████████| 6.23M/6.23M [00:06<00:00, 1.08MB/s]


In [5]:
# ~~ MiDaS - Depth Estimation ~~

model_type = "MiDaS_small"  # MiDaS v2.1 - Small   (lowest accuracy, highest inference speed)
#model_type = "DPT_Hybrid"   # MiDaS v3 - Hybrid    (medium accuracy, medium inference speed)
#model_type = "DPT_Large"     # MiDaS v3 - Large     

# Download model if not yet downloaded
midas = torch.hub.load("intel-isl/MiDaS", model_type)

# Set torch options
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()

# Load transforms for each model
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

# Select transforms
if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

Using cache found in C:\Users\HP/.cache\torch\hub\intel-isl_MiDaS_master


Loading weights:  None


Using cache found in C:\Users\HP/.cache\torch\hub\rwightman_gen-efficientnet-pytorch_master
Using cache found in C:\Users\HP/.cache\torch\hub\intel-isl_MiDaS_master


In [6]:
def yolov8(frame):
    results = yolo_model.predict(
        source = frame,
        
        conf = 0.5,
        iou = 0.7,
        show = False,
        save = False,
        retina_masks = True,
        classes = 0,
    )
    return results

def get_cropped_people(results, frame):
    '''
    results is from yolov8
    Returns list of cropped images of persons, and corresponding list of their top left coordinates
    '''
    im_list = []
    coordi_list = []
    area_list = []
    for result in results:
        for box in result.boxes:
            for a, b, c, d in box.xyxy:
                x1 = int(a)
                y1 = int(b)
                x2 = int(c)
                y2 = int(d)
            croppped = frame[y1:y2, x1:x2]
            area_list.append((x2-x1)*(y2-y1))
            im_list.append(croppped)
            
            '''x = (x1 + x2)/2
            y = (y1 + y2)/2'''
            coordi_list.append((x1, y1))
    return im_list, coordi_list, area_list

def get_cropped_faces(people_list, coordi_list, area_list):
    '''
    people_list is from get_cropped_people
    Returns list of cropped images of faces 
    & corrsponding list of bounding boxes in main frame
    '''
    face_list = []
    bounding_box_list = []
    new_area_list = []

    for i, person in enumerate(people_list):

        x_person, y_person = coordi_list[i]
        area = area_list[i]
 
        if person.shape[0] == 0 or person.shape[1] == 0 or person is None:
            continue
        try:       
            face_objs = FaceDetector.detect_faces(
                face_detector = face_detector,
                detector_backend = 'ssd',
                img = person,
                align = True,
            )
        except:
            continue

        
        if face_objs is None:
            print('No face detected')
            continue

        test_count = 0
        for im_obj in face_objs:
            face, box, conf = im_obj
            face_list.append(face)

            '''print('Face detected')
            
            print(box)
            print(test_count)
            test_count += 1'''

            
            '''cv.imshow('face', face)
            cv.waitKey(0)
            cv.destroyAllWindows()'''
            
            x1 = int(box[0])
            y1 = int(box[1])
            x2 = int(box[0] + box[2])
            y2 = int(box[1] + box[3])

            bounding_box_p1 = (
                x_person + x1,
                y_person + y1,
            )
            bounding_box_p2 = (
                x_person + x2,
                y_person + y2,
            )
            #print(bounding_box_p1, bounding_box_p2)
            bounding_box_list.append((bounding_box_p1, bounding_box_p2))

            #Area of the person-box from yolo, corresponding to this (i'th) face
            new_area_list.append(area)
            
    return face_list, bounding_box_list, new_area_list


def get_depth(img):
    input_batch = transform(img).to(device)
    with torch.no_grad():
        prediction = midas(input_batch)

        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode="bicubic",
            align_corners=False,
        ).squeeze()

    output = prediction.cpu().numpy()

    output /= output.max()
    output *= 255
    output = output.astype(np.uint8)
    output = cv.applyColorMap(output, cmapy.cmap('magma'))

    return output

def face_recognition(target_face, face_pic, model_name="Facenet", detector_backend="skip"):
    result = DeepFace.verify(
        target_face, 
        face_pic, 
        model_name, 
        detector_backend
        )
    return result

def findCosineDistance(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

'''
def cosine(emb1, emb2):
    cos = np.dot(emb1, emb2.T) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
    return cos'''

'\ndef cosine(emb1, emb2):\n    cos = np.dot(emb1, emb2.T) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))\n    return cos'

In [7]:

#Get target face~ 
target_face = cv.imread(target_face_path)

'''cv.imshow('target', target_face)
cv.waitKey(0)
cv.destroyAllWindows()

target_rep = target_embd[0]["embedding"]'''

'cv.imshow(\'target\', target_face)\ncv.waitKey(0)\ncv.destroyAllWindows()\n\ntarget_rep = target_embd[0]["embedding"]'

In [11]:
#Run the video, and go frame by frame ~
#cap = cv.VideoCapture('workshop-vid.mp4')
#cap = cv.VideoCapture('obama-biden.mp4')
#cap = cv.VideoCapture('obama-bgface.mp4')
#cap = cv.VideoCapture('obama-laterals-and-more.mp4')
#cap = cv.VideoCapture('obamawalk.mp4')
#cap = cv.VideoCapture(0)

cap = cv.VideoCapture(source_vid_path)

#FPS Calculation-----------------------------------------------------------------------------#
frame_count = 0
fps = 0

#Video writer-------------------------------------------------------------------------------#
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))*2
   
size = (frame_width, frame_height)

output = cv.VideoWriter(vid_save_path, 
                         cv.VideoWriter_fourcc(*'mp4v'),
                         5, size)
#--------------------------------------------------------------------------------------------#
previous_area = -1

#Main loop---------------------------------------------------------------------------------#
tic = time.time()
while True:
    
    ret, frame = cap.read()
    frame_count = frame_count + 1

    nextFrameNo = cap.get(cv.CAP_PROP_POS_FRAMES)
    totalFrames = cap.get(cv.CAP_PROP_FRAME_COUNT)
    complete = nextFrameNo/totalFrames
    print(complete*100, '%')


    if not ret:
        break

    if np.any(frame) == False:
        continue
    
    # Person detector---------------------------------------------------------------------------------#
    results = yolov8(frame)
    cropped_people, coordi_list, area_l = get_cropped_people(results, frame)
    output_frame = results[0].plot() 
    
    # Face detector------------------------------------------------------------------------------------#
    if len(cropped_people) != 0:
        face_list, bounding_boxes, person_areas = get_cropped_faces(cropped_people, coordi_list, area_l)
        
        #Boxes on faces
        for i, face in enumerate(face_list):
            p1, p2 = bounding_boxes[i]
            #print('Drawing box', p1, p2)
            cv.rectangle(output_frame, p1, p2, (0, 255, 255), 2)

        #Face recognition--------------------------------------------------------------------------------#
        face_model = "SFace"
        
        for i, face_pic in enumerate(face_list):
            face = face_pic.copy()
            if np.any(face) == False:
                continue

            try:
                result = face_recognition(
                    target_face,
                    face,
                    model_name = face_model,
                )
                print("using - ", face_model)
            except:
                continue
            

            # We found our guy ??????????????
            if (result['verified'] == True ):
                p1, p2 = bounding_boxes[i]
                cv.rectangle(output_frame, p1, p2, (255, 0, 20), 2)
                #cv.putText(output_frame, 'found u!', p1, cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 20), 2, cv.LINE_AA)

                if previous_area != -1:

                    diff = person_areas[i] - previous_area

                    threshold = frame.shape[0] * frame.shape[1] * sensitivity

                    if diff > threshold:
                        cv.putText(output_frame, 'Oh, You are approaching me?', p1, cv.FONT_HERSHEY_SIMPLEX, 1, (54, 255, 2), 2, cv.LINE_AA)
                    elif diff < -threshold:
                        cv.putText(output_frame, 'You going away?', p1, cv.FONT_HERSHEY_SIMPLEX, 1, (54, 255, 2), 2, cv.LINE_AA)
                    else:
                        cv.putText(output_frame, 'You not gonna move now?', p1, cv.FONT_HERSHEY_SIMPLEX, 1, (54, 255, 2), 2, cv.LINE_AA)
                        
                previous_area = person_areas[i]


            '''face_embd = DeepFace.represent(
                face,
                model_name = 'Facenet',
                detector_backend = 'skip',
                )
            
            face_rep = face_embd[0]["embedding"]

            cos_dist = findCosineDistance(target_rep, face_rep)'''


            '''#Display each face
                cv.imshow("Match", face_pic)
                
                if cv.waitKey(1) & 0xFF == ord('m'):
                    cap.release()
                    cv.destroyAllWindows()
                    exit()
                cv.waitKey(0)
                cv.destroyAllWindows()'''

#------------------------------------------------------------------------------------------------#
    #Depth map
    depth = get_depth(frame)
    #cv.imshow('depth', depth)
    output_frame = np.concatenate((output_frame, depth), axis=0)
#------------------------------------------------------------------------------------------------#
    #Output ~

    if frame_count%10 == 0:
        toc = time.time()
        fps = round((frame_count / (toc - tic)), 3)
        frame_count = 0
        tic = toc
        
    print('FPS = ', fps)
    
    cv.putText(
        output_frame,
        f'FPS: {fps}',
        (10, 50),
        cv.FONT_HERSHEY_SIMPLEX,
        1,
        (255, 0, 125),
        2,
        cv.LINE_AA,     
    )

    output.write(output_frame)
    cv.imshow("Output", output_frame)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break
    

cap.release()
output.release()
cv.destroyAllWindows()

-0.0 %


0: 480x640 1 person, 253.7ms
Speed: 0.0ms preprocess, 253.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)



FPS =  0
-0.0 %


0: 480x640 1 person, 225.4ms
Speed: 1.0ms preprocess, 225.4ms inference, 7.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 person, 157.3ms


FPS =  0
-0.0 %


...

FPS =  2.245
-0.0 %


0: 480x640 1 person, 190.3ms
Speed: 0.0ms preprocess, 190.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 person, 147.7ms


FPS =  2.245
-0.0 %


Speed: 4.0ms preprocess, 147.7ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 640)


FPS =  2.245
